In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model
import pickle

2023-10-04 15:55:39.580820: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('SMSSpamCollection.csv' , sep='\t',names=["cat","sms"])

In [5]:
X_test_mlp = np.load("X_test_mlp.npy", allow_pickle=True)
print(X_test_mlp)
X_test_tf = np.load("X_test_tf.npy", allow_pickle=True)
print(X_test_tf)

  (0, 7048)	0.1784900275985698
  (0, 4025)	0.19351337157220336
  (0, 3610)	0.21524949286062467
  (0, 1532)	0.12790243547505906
  (0, 1343)	0.9316665218271033
  (1, 7029)	0.2423922569115933
  (1, 6964)	0.3401716221276635
  (1, 6606)	0.08588657000857217
  (1, 6466)	0.3524324306350518
  (1, 6198)	0.18545699148314512
  (1, 4232)	0.1232219982816173
  (1, 4029)	0.31289675797039523
  (1, 3705)	0.324216849383398
  (1, 3628)	0.16976024793403102
  (1, 3491)	0.2090516696727675
  (1, 3475)	0.14818606288070016
  (1, 3400)	0.15575123077033073
  (1, 3356)	0.1765224371840541
  (1, 3306)	0.18825456703483828
  (1, 2923)	0.15629902585286376
  (1, 1843)	0.21121823398308234
  (1, 1693)	0.22868662838467882
  (1, 1161)	0.18394336982581413
  (1, 1154)	0.20568643105046533
  (1, 938)	0.11659365035539532
  :	:
  (1390, 244)	0.24044182671051856
  (1391, 7166)	0.15263279427393434
  (1391, 7105)	0.20230187065060928
  (1391, 6631)	0.21097549792756096
  (1391, 6606)	0.17728430518286584
  (1391, 6493)	0.23484656863608

In [4]:
tf_model = tf_model = load_model('modeleV1.h5')

with open('mlp.pkl', 'rb') as file:
        mlp_model = pickle.load(file)

/usr/local/Caskroom/miniconda/base/envs/spam_detection/lib/python3.11/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelBinarizer from version 1.3.1 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/spam_detection/lib/python3.11/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MLPClassifier from version 1.3.1 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
pred_prob_tf = tf_model.predict(X_test_tf)

44/44 [==============================] - 17s 374ms/step


In [8]:
pred_prob_tf.shape

(1393, 1)

In [12]:
pred_prob_mlp = mlp_model.predict_proba(X_test_mlp.item())

In [14]:
pred_prob_mlp.shape

(1393, 2)

In [ ]:
fpr1, tpr1, thresh1 = roc_curve(y_test, pred_prob_tf, pos_label=1)
fpr2, tpr2, thresh2 = roc_curve(y_test, pred_prob_mlp[:,1], pos_label=1)

In [ ]:
random_probs = [0 for i in range(len(y_test))]
p_fpr, ptpr,  = roc_curve(y_test, random_probs, pos_label=1)

In [ ]:
plt.plot(fpr1, tpr1, linestyle='--',color='orange', label='Logistic Regression')
plt.plot(fpr2, tpr2, linestyle='--',color='green', label='KNN')
plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')

plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.show()